# Drivers of Airbnb prices and recommendations -  Deriving pricing recommendations for landlords and Airbnb


The data set consists of 48895 listed Airbnb houses for which 16 columns of information are given. The information can be grouped into four distinct categories given by:

Host descriptors:

 - host_id: host id
 - host_name: name of the host
 - calculated_host_listings_count: amount of listing per host

Listing descriptors:

 - id: listing ID
 - name: name of the listing
 - room_type: listing space type
 - minimum_nights: amount of nights minimum
 - availability_365: number of days when listing is available for booking
 - price: price in dollars

Review descriptors:

 - number_of_reviews: number of reviews
 - last_review: latest review
 - reviews_per_month: number of reviews per month

Location descriptors:

 - neighbourhood_group: location
 - neighbourhood: area
 - latitude: latitude coordinates
 - longitude: longitude coordinates

As for any econometric analysis, diving cluelessly into regression analysis without examining the dataset and clarifying the objective first does not yield suitable results. Preprocessing the data in accordance with the goal to predict the price of the listings is the first step. In order to reduce the heterogeneity in the dataset, the following analysis focusses on offerings for short-term rentals (e.g. visitors to the city) with 30 minimum nights at most.

Based on this limitation factors that potentially influence the listing's price include but are not limited to :

 - neighbourhood
 - proximity to relevant infrastructure (e.g. subway) and landmarks
 - quality of the listing
 - reviews
 - type of listing (e.g. shared/ individual room)

While these factors are included in the basic dataset, other important characteristics are not consideres in this file. Hence, we utilized additional data sources and own calculations to enhance the analysis. Variables created with new datasets are the following:
 - minimum distance to the next subway station
 - minimum distance to the next popular landmarks
 - is the host a 'superhost'
 - can the listing can be referred to as 'premium'
 - review scores
 - is the host verified
 - property type

The remainder is structured as follows. First, we import all required packages and scan the data for outliers that would significantly impair the result of the prediction. In a second step, the visualization of different variables and the distribution is taken as an indicator whether or not the variables are normalized in the course of logarithmizing. In that regard, additional variables are created with the given and outside data to potentially enrich the analysis. After that, analyzing the correlation of potential independent variables with the dependent variables provides a first inside as to what is expected for the regression. Lastly, the regression is run with various models to optimize the model fit. As a result, the findings are converted into action-oriented recommendations for Airbnb and individual homeowners to maximize profits and implementation notes.

In [ ]:
# Disclaimer: Running the entire notebook at once is time consuming as the distance calculations entail 
# millions of individual equations. To reproduce the results on your own, either run the notebook as is 
# (this takes approximately 4 hours) or load the cache files 

## Imports

In [ ]:
# importing required packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from textacy import preprocessing
from gensim.parsing.preprocessing import remove_stopwords
import re
from collections import Counter
import geopy.distance
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# importing the basic dataset for the Airbnb listings

df = pd.read_csv(r'AB_NYC_2019.csv')

In [ ]:
# inspecting the columns of the dataset, its shape, and the first 5 entries to gain an understanding of the 
# structure of the data 

df.columns

In [ ]:
df.shape

In [ ]:
df.head()

### Additional Data

In [ ]:
# importing an extended dataset from http://insideairbnb.com/get-the-data.html to utilise data beyond the basic
# categories provided to have a more granular analysis

df_full = pd.read_csv(r'AB_NYC_2019 - Full Listings.csv')

In [ ]:
# appending relevant information from the extended dataset to the basic dataframe

df['host_is_superhost'] = df_full['host_is_superhost'].copy()
df['host_identity_verified'] = df_full['host_identity_verified'].copy()
df['review_scores_rating'] = df_full['review_scores_rating'].copy()
df['property_type'] = df_full['property_type'].copy()

In [ ]:
# later in the worksheet, two additional excel files that comprise the coordinates of subways stations
# and landmarks are loaded and added to the df

## Cleaning

Before deriving the feature columns which will be used to perform the different analyses, we clean the data by omitting rows based on certain criteria as defined below in the corresponding cells.

In [ ]:
# getting an understanding of missing values in the dataset

df.isnull().sum()

In [ ]:
# filling and dropping rows with empty cells in different columns 

df['reviews_per_month'].fillna(0,inplace=True)
df['review_scores_rating'].fillna(0,inplace=True)
df.dropna(subset=['price','name','host_name','latitude','longitude'], inplace=True)

In [ ]:
# the 'price' column contains outliers that are due to unreasonably prices listings, to enhance the precision
# of estimates - to do that, remove the top and bottom 2.5 percentiles

lower_bound = np.percentile(df['price'], 2.5)
upper_bound = np.percentile(df['price'], 97.5)
df = df[(df['price'] <= upper_bound) & (df['price'] > lower_bound)]

In [ ]:
# dropping entries with more than 1 review per day as it indicates flawful or unrealistic entries

df.drop(df[df['reviews_per_month'] >= 31].index, inplace=True)

In [ ]:
# dropping entries with minimum nights greater than 1 month - as stated in the introduction: by that, we 
# narrow down the scope of the analyses and focus on touristic travel.

df.drop(df[df['minimum_nights'] >= 31].index, inplace=True)


## Feature Engineering

In this section, we have a look at the existing columns and derive new feature columns from some of them.

### 'name': identifying premium listings

Below we defined a list of words contained in the descriptions of the most expensive listings. Such words would usually be associated with a 'premium' listing, i.e. a listing that should be more expensive than others.

In [ ]:
premium = ['luxury', 'terrace', '5-star', 'pool', 'loft', 'victorian', 'posh', 'NO GUEST SERVICE FEE', 'elegant', 'fantastical', 'penthouse', 'presidential', 'gym']
df['is_Premium'] = df['name'].apply(lambda x: any(feature in x for feature in premium)) 
df['is_Premium'] = df['is_Premium'].apply(lambda x: 1 if x == True else 0)

### 'name': popular words

Assuming that some words appear more frequently across the descriptions of listings on Airbnb, we first preprocess the descriptions and then derive a list of the top 25 most popular/frequent words. Subsequently, we one-hot encode this list.

In [ ]:
description_list = df['name'].tolist()

In [ ]:
# remove stopwords as these word do not contribute to a meaningful description of the listing

def preprocessor(lst): 
    processed_strings = [re.sub(r'\d+','', preprocessing.remove_punctuation(str(lst[i]))) for i in range(len(lst))] 
    lstx = [remove_stopwords(str(processed_strings[i])) for i in range(len(processed_strings))] 
    return lstx

In [ ]:
# applying the preprocessor function to the list of listing descriptions

proc = preprocessor(description_list)

In [ ]:
# defining a function to split strings

def split_name(name):
    spl = str(name).split()
    return spl

In [ ]:
# getting the word count

_names_for_count_=[]

for x in proc:
    for word in split_name(x):
        word=word.lower()
        _names_for_count_.append(word)

In [ ]:
# getting the top 25 most used words

top25 = Counter(_names_for_count_).most_common()
top25 = top25[0:25]

In [ ]:
# using a barplot to visualize the most frequently used words

sub_w = pd.DataFrame(top25)
sub_w.rename(columns={0:'Words', 1:'Count'}, inplace=True)

plot = sns.barplot(x='Words', y='Count', data=sub_w)
plot.set_title('Counts of the top 25 used words for listing names')
plot.set_ylabel('Count of words')
plot.set_xlabel('Words')
plot.set_xticklabels(plot.get_xticklabels(), rotation=80)

In [ ]:
# converting the list of most frequently used words to one-hot encoded columns of the dataframe

test_words = pd.DataFrame(top25)[0]

for word in test_words:
    word = word
    df[word] = df['name'].str.contains(word)
    df[word] = df[word].apply(lambda x: 1 if x == True else 0)

### 'last review': identifying activity

In this section, we derived new feature columns by taking the time since the last review a listing received into account. As can be seen below, we used to latest date in the dataset as the index for our review activity calculations.

In [ ]:
# date format yyyy-mm-dd

df['last_review'] = pd.to_datetime(df['last_review'])

In [ ]:
# latest review in the dataset

day_0 = df.last_review.max()

In [ ]:
# defining the different timespan variables based on day_0

one_week = day_0 - pd.Timedelta(days=7)
one_month = day_0 - pd.Timedelta(days=31)
one_year = day_0 - pd.Timedelta(days=365)
five_years = day_0 - pd.Timedelta(days=(365*5))

In [ ]:
# one-hot encoding the review activity variables and appending the data to the dataframe

df['last_week'] = df['last_review'].apply(lambda x: 1 if x > one_week else 0)
df['last_month'] = df['last_review'].apply(lambda x: 1 if x <= one_week and x > one_month else 0)
df['last_year'] = df['last_review'].apply(lambda x: 1 if x <= one_month and x > one_year else 0)
df['last_five_years'] = df['last_review'].apply(lambda x: 1 if x <= five_years else 0)
df['no_review'] = df['last_review'].isnull().astype('int')

In [ ]:
# deleting the last_review column as it is of no use anymore

del df['last_review']

### 'room type': encoding different types

Given that we identified different types of rooms offered on Airbnb, we decided to also one-hot encode these different types and include appropriate feature columns in our dataset.

In [ ]:
# getting columns with dummies for room types

room_types = pd.get_dummies(df['room_type'], prefix='is')
df = pd.concat([df, room_types], axis=1)

### 'neighbourhood': encoding different neighbourhoods

We applied same procedure that was used to derive the dummy variables for the different room types to the different neighbourhoods of New York City.

In [ ]:
# getting columns with dummies for neighbourhood

neighborhood = pd.get_dummies(df['neighbourhood'], prefix='nhood')
df = pd.concat([df, neighborhood], axis=1)

### 'subway station proximity': calculate minimum distance

To go beyond the data provided on moodle and on insideairbnb.com, we concluded that, for tourists, which are presumably the most frequent users of Airbnb, there are two further aspects that are usually taken into account: the distance of a place to the nearest subway or bus and the distance to popular landmarks. As a result, we again drew on external data and made use of the latitude and longitude data by calculating the aforementioned distances for each listing. The raw data can be found under the following URL: https://data.ny.gov/widgets/i9wp-a4ja . 

leave the following uncommented when distances should be taken from excel to avoid excessive runtime

In [ ]:
min_distances_sub = pd.read_excel(r'sub_dist.xlsx')
min_distances_sub = min_distances_sub.drop(['Unnamed: 0'], axis=1).reset_index()
min_distances_sub = min_distances_sub[0].tolist()

uncomment the following when distances should be calculated (time-intensive)

In [ ]:
'''

# importing data on the location of subway stations in NYC
subways = pd.read_excel(r'NYC_Transit_Subway_Entrance_And_Exit_Data.xlsx')

# defining a distance function using geopy to return distance in meters
def distance(coord1, coord2):
    return geopy.distance.distance(coord1, coord2).m

# converting the latitude and longitude data of the listings to a list of tuples
listings_lat = df.latitude.tolist()
listings_long = df.longitude.tolist()

lst_lat_long = list(zip(listings_lat, listings_long))

# getting unique station locations and converting coordinates to a list of tuples 
station_location = subways['Station Location'].tolist()
unique_stations = set(station_location)
unique_stations_location = [tuple(eval(station)) for station in unique_stations]

# note that executing this cell will require several hours as around 20 million calculations are performed
final = []
for station in subways_location:
    temp = []
    for listing in lst_lat_long:
        temp.append(distance(station, listing))
    final.append(temp)

# transposing the list of subway distances to get it into the right shape
final_array_sub = np.array(final)
transpose = final_array_sub.T
transpose_list_sub = transpose.tolist()

# calculating the minimal subway distance for each listing
min_distances_sub = [min(transpose_list_sub[i]) for i in range(len(transpose_list_sub))]

'''

the following is to remain uncommented in either way

In [ ]:
# appending the distance between each listing and the closest subway station to the dataframe

df['min_distances_subway'] = min_distances_sub

### 'landmark' proximity: calculate minimum distance

Below the same procedure applies as above for calculating the minimal subway station distances. The raw data with coordinates for the landmarks can be found under the following URL: https://en.wikipedia.org/wiki/List_of_National_Historic_Landmarks_in_New_York_City . 

leave the following uncommented when distances should be taken from excel to avoid excessive runtime

In [ ]:
min_distances_landmarks = pd.read_excel(r'landm_dist.xlsx')
min_distances_landmarks = min_distances_landmarks.drop(['Unnamed: 0'], axis=1).reset_index()
min_distances_landmarks = min_distances_landmarks[0].tolist()

uncomment the following when distances should be calculated (time-intensive)

In [ ]:
'''

# importing data on the location of popular landmarks in NYC
landmarks = pd.read_excel(r'NYC_Landmark_Coordinates_filtered.xlsx')

# defining a distance function using geopy to return distance in meters
def distance(coord1, coord2):
    return geopy.distance.distance(coord1, coord2).m

# converting the latitude and longitude data of the landmarks to a list of tuples
landmarks_lat = landmarks.y.tolist()
landmarks_long = landmarks.x.tolist()

# converting the latitude and longitude data of the listings to a list of tuples
listings_lat = df.latitude.tolist()
listings_long = df.longitude.tolist()

lst_lat_long = list(zip(listings_lat, listings_long))

landmarks_lat_long = list(zip(landmarks_lat, landmarks_long))

# note that executing this cell will require several hours
final2 = []
for landmark in landmarks_lat_long:
    temp2 = []
    for listing in lst_lat_long:
        temp2.append(distance(landmark, listing))
    final2.append(temp2)

# transposing the list of landmark distances to get it into the right shape
final_array_landm = np.array(final2)
transpose2 = final_array_landm_sub.T
transpose_list_landm = transpose2.tolist()

# calculating the minimal landmark distance for each listing
min_distances_landmarks = [min(transpose_list_landm[i]) for i in range(len(transpose_list_landm))]

'''

the following is to remain uncommented in either way

In [ ]:
# appending the distance between each listing and the closest landmark to the dataframe

df['min_distance_landmark'] = min_distances_landmarks

### 'superhost': encode whether host is declared as a superhost

We assume that whether or not a host has 'superhost' status is a determinant for a listing's price and one-hot encode this variable.

In [ ]:
df['is_superhost'] = df['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

In [ ]:
del df['host_is_superhost']

### 'verified host': encode whether a host is verified

We assume that whether or not a host is verified is a determinant for a listing's price and one-hot encode this variable.

In [ ]:
df['has_identity_verified'] = df['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)

In [ ]:
del df['host_identity_verified']

### 'property type': encode the property type

We assume that a listing's property type is a determinant for a listing's price and one-hot encode this variable.

In [ ]:
property_types = pd.get_dummies(df['property_type'], prefix='is')
df = pd.concat([df, property_types ], axis=1)

In [ ]:
del df['property_type']

### final clean-up

Below we delete all further columns that will not be of any use for the analytical part.

In [ ]:
del df['id']
del df['name']
del df['host_id']
del df['host_name']
del df['neighbourhood_group']
del df['neighbourhood']
del df['room_type']

## Data visualization

Before proceeding to the analytical part, we look at the numerical variables' distributions in order to determine whether or not transforming some variables is necessary.

In [ ]:
fig = plt.figure(figsize=(14,14))
ax1 = fig.add_subplot(3, 3, 1)
ax2 = fig.add_subplot(3, 3, 2)
ax3 = fig.add_subplot(3, 3, 3)
ax4 = fig.add_subplot(3, 3, 4)
ax5 = fig.add_subplot(3, 3, 5)
ax6 = fig.add_subplot(3, 3, 6)
ax7 = fig.add_subplot(3, 3, 7)
ax8 = fig.add_subplot(3, 3, 8)
ax9 = fig.add_subplot(3, 3, 9)

ax1.hist(df['price'], bins=30)
ax1.set_ylabel("Frequency")
ax1.set_title('price')

ax2.hist(df['minimum_nights'], bins=30)
ax2.set_ylabel("Frequency")
ax2.set_title('minimum_nights')

ax3.hist((df['reviews_per_month']), bins=30)
ax3.set_ylabel("Frequency")
ax3.set_title('reviews_per_month')

ax4.hist(df['number_of_reviews'], bins=30)
ax4.set_ylabel("Frequency")
ax4.set_title('number_of_reviews')

ax5.hist(df['availability_365'], bins=30)
ax5.set_ylabel("Frequency")
ax5.set_title('availability_365')

ax6.hist(df['latitude'], bins=30)
ax6.set_ylabel("Frequency")
ax6.set_title('latitude')

ax7.hist(df['longitude'], bins=30)
ax7.set_ylabel("Frequency")
ax7.set_title('longitude')

ax8.hist(df['calculated_host_listings_count'], bins=30)
ax8.set_ylabel("Frequency")
ax8.set_title('calculated_host_listings_count')

ax9.hist(df['review_scores_rating'], bins=30)
ax9.set_ylabel("Frequency")
ax9.set_title('review_scores_rating')

plt.show()

Looking at the distributions, clearly the following are heavily skewed:

 - price
 - minimum_nights
 - reviews_per_month
 - number_of_reviews
 - availability_365
 - calculated_host_lising_count
 - review_scores_rating

One way to reduce the skeweness is to logarithmically transform the distributions:

In [0]:
fig = plt.figure(figsize=(14,14))
ax1 = fig.add_subplot(3, 3, 1)
ax2 = fig.add_subplot(3, 3, 2)
ax3 = fig.add_subplot(3, 3, 3)
ax4 = fig.add_subplot(3, 3, 4)
ax5 = fig.add_subplot(3, 3, 5)
ax6 = fig.add_subplot(3, 3, 6)
ax7 = fig.add_subplot(3, 3, 7)
ax8 = fig.add_subplot(3, 3, 8)
ax9 = fig.add_subplot(3, 3, 9)

ax1.hist(df['price'].apply(lambda x: np.log(x)), bins=30)
ax1.set_ylabel("Frequency")
ax1.set_title('price')

ax2.hist(df['minimum_nights'].apply(lambda x: np.log(x)), bins=30)
ax2.set_ylabel("Frequency")
ax2.set_title('minimum_nights')

ax3.hist((df['reviews_per_month']+1).apply(lambda x: np.log(x)), bins=30)
ax3.set_ylabel("Frequency")
ax3.set_title('reviews_per_month')

ax4.hist((df['number_of_reviews']+1).apply(lambda x: np.log(x)), bins=30)
ax4.set_ylabel("Frequency")
ax4.set_title('number_of_reviews')

ax5.hist((df['availability_365']+1).apply(lambda x: np.log(x)), bins=30)
ax5.set_ylabel("Frequency")
ax5.set_title('availability_365')

ax6.hist(df['latitude'], bins=30)
ax6.set_ylabel("Frequency")
ax6.set_title('latitude')

ax7.hist(df['longitude'], bins=30)
ax7.set_ylabel("Frequency")
ax7.set_title('longitude')

ax8.hist(df['calculated_host_listings_count'].apply(lambda x: np.log(x)), bins=30)
ax8.set_ylabel("Frequency")
ax8.set_title('calculated_host_listings_count')

ax9.hist((df['review_scores_rating']+1).apply(lambda x: np.log(x)), bins=30)
ax9.set_ylabel("Frequency")
ax9.set_title('review_scores_rating')

plt.show()

In [0]:
# the plot below indicates where the most expensive listings are located geographically
# as it seems, the district of Manhattan appears to have the most expensive listings

import urllib
plt.figure(figsize=(10,8))
i=urllib.request.urlopen('https://upload.wikimedia.org/wikipedia/commons/e/ec/Neighbourhoods_New_York_City_Map.PNG')
nyc_img=plt.imread(i)
plt.imshow(nyc_img,zorder=0,extent=[-74.258, -73.7, 40.49,40.92])
ax=plt.gca()
df_log.plot(kind='scatter', x='longitude', y='latitude', c='log_price', ax=ax, 
           cmap=plt.get_cmap('jet'), colorbar=True, alpha=0.4, zorder=5)
plt.legend()
plt.show()

## Data transformation

Here, we create a new dataframe, df_log, which contains the transformed numerical variables as feature columns.

In [ ]:
df_log = df.copy()

In [ ]:
df_log['price'] = (df_log['price']).apply(lambda x: np.log(x))
df_log['minimum_nights'] = (df_log['minimum_nights']+1).apply(lambda x: np.log(x))
df_log['reviews_per_month'] = (df_log['reviews_per_month']+1).apply(lambda x: np.log(x))
df_log['number_of_reviews'] = (df_log['number_of_reviews']+1).apply(lambda x: np.log(x))
df_log['availability_365'] = (df_log['availability_365']+1).apply(lambda x: np.log(x))
df_log['calculated_host_listings_count'] = (df_log['calculated_host_listings_count']+1).apply(lambda x: np.log(x))
df_log['review_scores_rating'] = (df_log['review_scores_rating']+1).apply(lambda x: np.log(x))

In [ ]:
df_log = df_log.rename(columns={'price': 'log_price', 'minimum_nights': 'log_minimum_nights', 'reviews_per_month':'log_reviews_per_month','number_of_reviews':'log_number_of_reviews','availability_365':'log_availability_365','calculated_host_listings_count':'log_calculated_host_listings_count','review_scores_rating':'log_review_scores_rating'})
df_log.head()

## Preparation of regression input (independent and dependent variables with training data respectively)

Below we define the different x and y variables which are subsequently to be used for the analyses. Here, we also consider the possibility of a dummy variable trap occurring and delete selected columns accordingly.

In [ ]:
x = df.drop(['no_review', 'is_Shared room', 'nhood_Allerton', 'price'], axis = 1).values
y = df['price'].values

In [ ]:
x_log = df_log.drop(['no_review', 'is_Shared room', 'nhood_Allerton', 'log_price'], axis = 1).values
y_log = df_log['log_price'].values

### room type split

To be even more granular, we also introduce different cases based on the room type. We perform this split as a result of the observation that the room type in fact seems to be a - relatively speaking - important feature.

In [ ]:
# creating separate dataframes where the room type only equals one specific variant

df_log_shared_room = df_log[df_log['is_Shared room'] == 1].reset_index()
df_log_private_room = df_log[df_log['is_Private room'] == 1].reset_index()
df_log_entire_home = df_log[df_log['is_Entire home/apt'] == 1].reset_index()

In [ ]:
x_log_shared_room = df_log_shared_room.drop(['no_review', 'is_Entire home/apt', 'is_Private room', 'is_Shared room','nhood_Allerton', 'log_price'], axis=1).values
y_log_shared_room = df_log_shared_room['log_price'].values

In [ ]:
x_log_private_room = df_log_private_room.drop(['no_review', 'is_Entire home/apt', 'is_Private room', 'is_Shared room','nhood_Allerton', 'log_price'], axis = 1).values
y_log_private_room = df_log_private_room['log_price'].values

In [ ]:
x_log_entire_home = df_log_entire_home.drop(['no_review', 'is_Entire home/apt', 'is_Private room', 'is_Shared room', 'nhood_Allerton', 'log_price'], axis = 1).values
y_log_entire_home = df_log_entire_home['log_price'].values

## ML Implementation

In [0]:
# Disclaimer: due to the large number of columns and rows, the execution of complex machine learning 
# algortihms is time consuming. We identified Model 2 (logarithmized variables and dummies to differentiate
# house types) to yield the best results based on the rmse. Hence, Model 1, Model 3, Model 4, and Model 5 
# are commented out in the following to enhance timely reproduction of the results. In case the reader is 
# interested in running the calculations, please remove the " ``` " at the beginning and end of that cell.

In [ ]:
# importing packages for ML implementation

import xgboost as xgb
import json

from sklearn.tree            import DecisionTreeRegressor
from sklearn.neural_network  import MLPRegressor
from sklearn.linear_model    import LinearRegression, Lasso, Ridge
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics         import mean_squared_error
from sklearn.metrics         import r2_score, make_scorer, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm             import SVR
from sklearn.utils           import shuffle
from tqdm                    import tqdm as progress_bar


In [ ]:
# setting random state to 42

random = np.random.RandomState(42)

In [ ]:
# splitting the different variables into training and test set

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=random)
x_log_train, x_log_test, y_log_train, y_log_test = train_test_split(x_log, y_log, test_size=0.2, random_state=random)
x_log_shared_room_train, x_log_shared_room_test, y_log_shared_room_train, y_log_shared_room_test = train_test_split(x_log_shared_room, y_log_shared_room, test_size=0.2, random_state=random)
x_log_private_room_train, x_log_private_room_test, y_log_private_room_train, y_log_private_room_test = train_test_split(x_log_private_room, y_log_private_room, test_size=0.2, random_state=random)
x_log_entire_home_train, x_log_entire_home_test, y_log_entire_home_train, y_log_entire_home_test = train_test_split(x_log_entire_home, y_log_entire_home, test_size=0.2, random_state=random)


In [ ]:
def cross_validation(X, y, *, model, k=10, log=False, desc=None):
    """Perform a k-fold cross validation."""
    rmse = []
    # Iterate over the k folds.
    for train, test in progress_bar(KFold(n_splits=k).split(X), desc=desc, total=k):
        model.fit(X[train], y[train])
        y_pred = model.predict(X[test])
        # If the sales price is provided on a log scale,
        # take the exponent first so that scores and
        # errors are comparable to the non-logged counterparts.
        if log:
            y_true, y_pred = np.exp(y[test]), np.exp(y_pred) 
        else:
            y_true, y_pred = y[test], y_pred
        # Collect the scores/errors for each fold.
        rmse.append(mean_squared_error(y_true, y_pred))
    # Round for convenience.
    return {
        np.round(np.sqrt(np.mean(rmse)),4)
    }
    

In [ ]:
rmse_results = {}

In [0]:
# Let us emphasize again that Model 1, Model 3, Model 4, and Model 5 in the following regression analyses
# are commented out to enhance timely reproduction of the results. In case the reader is interested in 
# running the calculations, please remove the " ``` " at the beginning and end of that cell.

### Linear regression

In [0]:
'''

# first model with standard variables without logarithmization and room types as dummy variables

model_LR1 = LinearRegression()
model_LR1.fit(x_train, y_train)
y_pred = model_LR1.predict(x_test)
rmse_LR1 = np.sqrt(mean_squared_error(y_test, y_pred))

rmse_results[("Linear regression", "Model 1")] = rmse_LR1

print(np.sqrt(mean_squared_error(y_test, y_pred)))

'''


In [ ]:
# second model with logarithmized variables and room types as dummy variables

model_LR2 = LinearRegression()
model_LR2.fit(x_log_train, y_log_train)
y_log_pred = model_LR2.predict(x_log_test)
rmse_LR2 = np.sqrt(mean_squared_error(y_log_test, y_log_pred))

rmse_results[("Linear regression", "Model 2")] = rmse_LR2

print(np.sqrt(mean_squared_error(y_log_test, y_log_pred)))

In [0]:
'''

# third model solely for shared rooms 

model_LR3 = LinearRegression()
model_LR3.fit(x_log_shared_room_train, y_log_shared_room_train)
y_log_shared_room_pred = model_LR3.predict(x_log_shared_room_test)
rmse_LR3 = np.sqrt(mean_squared_error(y_log_shared_room_test, y_log_shared_room_pred))

rmse_results[("Linear regression", "Model 3")] = rmse_LR3

print(np.sqrt(mean_squared_error(y_log_shared_room_test, y_log_shared_room_pred)))

'''

In [0]:
'''

# fourth model solely for private rooms 

model_LR4 = LinearRegression()
model_LR4.fit(x_log_private_room_train, y_log_private_room_train)
y_log_private_room_pred = model_LR4.predict(x_log_private_room_test)
rmse_LR4 = np.sqrt(mean_squared_error(y_log_private_room_test, y_log_private_room_pred))

rmse_results[("Linear regression", "Model 4")] = rmse_LR4

print(np.sqrt(mean_squared_error(y_log_private_room_test, y_log_private_room_pred)))

'''

In [0]:
'''

# fifth model solely for entire apartments 

model_LR5 = LinearRegression()
model_LR5.fit(x_log_entire_home_train, y_log_entire_home_train)
y_log_entire_home_pred = model_LR5.predict(x_log_entire_home_test)
rmse_LR5 = np.sqrt(mean_squared_error(y_log_entire_home_test, y_log_entire_home_pred))

rmse_results[("Linear regression", "Model 5")] = rmse_LR5

print(np.sqrt(mean_squared_error(y_log_entire_home_test, y_log_entire_home_pred)))

'''

### Ridge regression

In [ ]:
grid_search = GridSearchCV(
    estimator=Ridge(),
    param_grid={"alpha": [2 ** x for x in range(-8, 4)] + list(range(12, 65, 4))},
    cv=KFold(n_splits=4),
    n_jobs=-1,
)

In [0]:
'''

# first model with standard variables without logarithmization and room types as dummy variables

grid_search.fit(x, y)
alpha = grid_search.best_params_["alpha"]

rmse_results[("ridge", "Model 1")] = cross_validation(x, y, model=Ridge(alpha=alpha))
rmse_results[("ridge", "Model 1")]

'''

In [ ]:
# second model with logarithmized variables and room types as dummy variables

grid_search.fit(x_log, y_log)
alpha = grid_search.best_params_["alpha"]

rmse_results[("ridge", "Model 2")] = cross_validation(x_log, y_log, model=Ridge(alpha=alpha))
rmse_results[("ridge", "Model 2")]

In [0]:
'''

# third model solely for shared rooms 

grid_search.fit(x_log_shared_room, y_log_shared_room)
alpha = grid_search.best_params_["alpha"]

rmse_results[("ridge", "Model 3")] = cross_validation(x_log_shared_room, y_log_shared_room, model=Ridge(alpha=alpha))
rmse_results[("ridge", "Model 3")]

'''

In [0]:
'''

# fourth model solely for private rooms 

grid_search.fit(x_log_shared_room, y_log_shared_room)
alpha = grid_search.best_params_["alpha"]

rmse_results[("ridge", "Model 4")] = cross_validation(x_log_private_room, y_log_private_room, model=Ridge(alpha=alpha))
rmse_results[("ridge", "Model 4")]

'''

In [0]:
'''

# fifth model solely for entire homes 

grid_search.fit(x_log_entire_home, y_log_entire_home)
alpha = grid_search.best_params_["alpha"]

rmse_results[("ridge", "Model 5")] = cross_validation(x_log_entire_home, y_log_entire_home, model=Ridge(alpha=alpha))
rmse_results[("ridge", "Model 5")]

'''

### Lasso regression

In [ ]:
tol = 0.1
grid_search = GridSearchCV(
    estimator=Lasso(tol=tol, selection="random", random_state=random),
    param_grid={"alpha": [2 ** x for x in range(-8, 4)] + list(range(12, 65, 4))},
    cv=KFold(n_splits=4),
    n_jobs=-1,
)

In [0]:
'''

# first model with standard variables without logarithmization and room types as dummy variables

grid_search.fit(x, y)
alpha = grid_search.best_params_["alpha"]

rmse_results[("lasso", "Model 1")] = cross_validation(x, y, model=Lasso(alpha=alpha, tol=tol))
rmse_results[("lasso", "Model 1")]

'''

In [ ]:
# second model with logarithmized variables and room types as dummy variables

grid_search.fit(x_log, y_log)
alpha = grid_search.best_params_["alpha"]

rmse_results[("lasso", "Model 2")] = cross_validation(x_log, y_log, model=Lasso(alpha=alpha, tol=tol))
rmse_results[("lasso", "Model 2")]

In [0]:
'''

# third model solely for shared rooms 

grid_search.fit(x_log_shared_room, y_log_shared_room)
alpha = grid_search.best_params_["alpha"]

rmse_results[("lasso", "Model 3")] = cross_validation(x_log_shared_room, y_log_shared_room, model=Lasso(alpha=alpha, tol=tol))
rmse_results[("lasso", "Model 3")]

'''

In [0]:
'''

# fourth model solely for private rooms 

grid_search.fit(x_log_private_room, y_log_private_room)
alpha = grid_search.best_params_["alpha"]

rmse_results[("lasso", "Model 4")] = cross_validation(x_log_private_room, y_log_private_room, model=Lasso(alpha=alpha, tol=tol))
rmse_results[("lasso", "Model 4")]

'''

In [0]:
'''

# fifth model solely for entire homes 

grid_search.fit(x_log_entire_home, y_log_entire_home)
alpha = grid_search.best_params_["alpha"]

rmse_results[("lasso", "Model 5")] = cross_validation(x_log_entire_home, y_log_entire_home, model=Lasso(alpha=alpha, tol=tol))
rmse_results[("lasso", "Model 5")]

'''

### XGBoost

In [0]:
'''

# first model with standard variables without logarithmization and room types as dummy variables

model_XGB1 = xgb.XGBRegressor()
model_XGB1.fit(x_train, y_train)
y_XGB_pred = model_XGB1.predict(x_test)
rmse_XGB1 = np.sqrt(mean_squared_error(y_test, y_XGB_pred))

rmse_results[("XGBoost", "Model 1")] = rmse_XGB1

print(rmse_XGB1)

'''

In [ ]:
# second model with logarithmized variables and room types as dummy variables

model_XGB2 = xgb.XGBRegressor()
model_XGB2.fit(x_log_train, y_log_train)
y_log_XGB_pred = model_XGB2.predict(x_log_test)
rmse_XGB2 = np.sqrt(mean_squared_error(y_log_test, y_log_XGB_pred))

rmse_results[("XGBoost", "Model 2")] = rmse_XGB2

print(rmse_XGB2)

In [0]:
'''

# third model solely for shared rooms

model_XGB3 = xgb.XGBRegressor()
model_XGB3.fit(x_log_shared_room_train, y_log_shared_room_train)
y_log_XGB_shared_room_pred = model_XGB3.predict(x_log_shared_room_test)
rmse_XGB3 = np.sqrt(mean_squared_error(y_log_shared_room_test, y_log_XGB_shared_room_pred))

rmse_results[("XGBoost", "Model 3")] = rmse_XGB3

print(rmse_XGB3)

'''

In [0]:
'''

# fourth model solely for private rooms 

model_XGB4 = xgb.XGBRegressor()
model_XGB4.fit(x_log_private_room_train, y_log_private_room_train)
y_log_XGB_private_room_pred = model_XGB4.predict(x_log_private_room_test)
rmse_XGB4 = np.sqrt(mean_squared_error(y_log_private_room_test, y_log_XGB_private_room_pred))

rmse_results[("XGBoost", "Model 4")] = rmse_XGB4

print(rmse_XGB4)

'''

In [0]:
'''

# fifth model solely for entire homes 

model_XGB5 = xgb.XGBRegressor()
model_XGB5.fit(x_log_entire_home_train, y_log_entire_home_train)
y_log_XGB_entire_home_pred = model_XGB5.predict(x_log_entire_home_test)
rmse_XGB5 = np.sqrt(mean_squared_error(y_log_entire_home_test, y_log_XGB_entire_home_pred))

rmse_results[("XGBoost", "Model 5")] = rmse_XGB5

print(rmse_XGB5)

'''

### Support Vector regression

In [0]:
'''

# first model with standard variables without logarithmization and room types as dummy variables

model_SVR1 = SVR()
model_SVR1.fit(x_train, y_train)
y_SVR_pred = model_SVR1.predict(x_test)
rmse_SVR1 = np.sqrt(mean_squared_error(y_test, y_SVR_pred))

rmse_results[("SVR", "Model 1")] = rmse_SVR1

print(rmse_SVR1)

'''

In [0]:
# second model with logarithmized variables and room types as dummy variables

model_SVR2 = SVR()
model_SVR2.fit(x_log_train, y_log_train)
y_log_SVR_pred = model_SVR2.predict(x_log_test)
rmse_SVR2 = np.sqrt(mean_squared_error(y_log_test, y_log_SVR_pred))

rmse_results[("SVR", "Model 2")] = rmse_SVR2

print(rmse_SVR2)

In [0]:
'''

# third model solely for shared rooms

model_SVR3 = SVR()
model_SVR3.fit(x_log_shared_room_train, y_log_shared_room_train)
y_log_SVR_shared_room_pred = model_SVR3.predict(x_log_shared_room_test)
rmse_SVR3 = np.sqrt(mean_squared_error(y_log_shared_room_test, y_log_SVR_shared_room_pred))

rmse_results[("SVR", "Model 3")] = rmse_SVR3

print(rmse_SVR3)

'''

In [0]:
'''

# fourth model solely for private rooms 

model_SVR4 = SVR()
model_SVR4.fit(x_log_private_room_train, y_log_private_room_train)
y_log_SVR_private_room_pred = model_SVR4.predict(x_log_private_room_test)
rmse_SVR4 = np.sqrt(mean_squared_error(y_log_private_room_test, y_log_SVR_private_room_pred))

rmse_results[("SVR", "Model 4")] = rmse_SVR4

print(rmse_SVR4)

'''

In [0]:
'''

# fifth model solely for entire homes 

model_SVR5 = SVR()
model_SVR5.fit(x_log_entire_home_train, y_log_entire_home_train)
y_log_SVR_entire_home_pred = model_SVR5.predict(x_log_entire_home_test)
rmse_SVR5 = np.sqrt(mean_squared_error(y_log_entire_home_test, y_log_SVR_entire_home_pred))

rmse_results[("SVR", "Model 5")] = rmse_SVR5

print(rmse_SVR5)

'''

### Random Forest regression

In [0]:
'''

rf1 = RandomForestRegressor(
    n_estimators=500,
    n_jobs=-1, random_state=random
)

rmse_results[("rf", "Model 1")] = cross_validation(x, y, model=rf1)
rmse_results[("rf", "Model 1")]

'''

In [0]:
rf2 = RandomForestRegressor(
    n_estimators=500,
    n_jobs=-1, random_state=random
)

rmse_results[("rf", "Model 2")] = cross_validation(x_log, y_log, model=rf2)
rmse_results[("rf", "Model 2")]

In [0]:
'''

rf3 = RandomForestRegressor(
    n_estimators=500,
    n_jobs=-1, random_state=random
)

rmse_results[("rf", "Model 3")] = cross_validation(x_log_shared_room, y_log_shared_room, model=rf3)
rmse_results[("rf", "Model 3")]

'''

In [0]:
'''

rf4 = RandomForestRegressor(
    n_estimators=500,
    n_jobs=-1, random_state=random
)

rmse_results[("rf", "Model 4")] = cross_validation(x_log_private_room, y_log_private_room, model=rf4)
rmse_results[("rf", "Model 4")]

'''

In [0]:
'''

rf5 = RandomForestRegressor(
    n_estimators=500,
    n_jobs=-1, random_state=random
)

rmse_results[("rf", "Model 5")] = cross_validation(x_log_entire_home, y_log_entire_home, model=rf5)
rmse_results[("rf", "Model 5")]

'''

### Neural network

Since we have a considerable amount of data points, we decided to experiment with different setups, ranging from minimally small setups with just one or two layers and equally few neurons to arbitrarily large networks. Ultimately, it seems as if a relatively large architecture yields better results, presumably given the large number of input features. The smallest RMSE we managed to achieve was 0.1635 with a setup of 15 layers and a gradually decreasing number of neurons per layer. This result, however, likely is the result of overfitting the model as this initial setup did not include a validation split. Including a validation split, the model below (model1) yielded the lowest RMSE (0.3) of the setups we experimented with. Additionally, we tried using MLPRegressor, which, however, only yielded an RMSE of around 0.37.

In [0]:
import keras
import tensorflow

In [0]:
# number of hidden layers nh = Ns/(α∗ (Ni + No))
# Ns: samples
# No: output neurons
# Ni: input neurons
# α arbitrary scaling factor


model2 = keras.Sequential([

    keras.layers.Dense(295, activation='relu', input_shape=(295,)),
    
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),

    keras.layers.Dense(1, activation='relu')

])


In [0]:
model2.compile(optimizer='adam', loss='mean_squared_error')

In [0]:
# fitting Model 2 variables

model2.fit(x_log_train, y_log_train, epochs=100, batch_size=500, validation_split=0.1, callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [0]:
NN2_y_log_pred = model2.predict(x_log_test)
rmse_NN2 = np.sqrt(mean_squared_error(y_log_test, NN2_y_log_pred))

rmse_results[("Neural Network", "Model 2")] = rmse_NN2

print(rmse_NN2) 

In [0]:
model2.save('model2')

## Exporting final RMSE scores to Excel

In [0]:
final_scores = pd.DataFrame.from_dict(rmse_results, orient='index')
final_scores.to_excel(r'final_scores.xlsx')

## Recommendations

Based on the regression results, we identified several levers to positively influence the listings‘s price and therewith maximize returns of Airbnb and the homeowners. For the purpose of the challenge, the guiding principle for the selection of recommendations is profit maximization. It is needless to say that other factors such as sociability also contribute to the welfare of homeowners. Hence, factors beyond the financial impact would need to be considered despite the analysis. 

Starting with homeowners, the recommendation with the highest financial benefit attached to it is to offer entire homes instead of private/shared rooms only. For all machine learning algorithms, the home type was by far the most important determinant. Especially tourists seem to value more amenity and flexibility, and are willing to pay more for respective homes. Secondly, the data suggests that the superhost status has a positive impact on the price charged for a home. Hence, we recommend that homeowners should actively work towards meeting the superhost criteria. The criteria for superhosts are a rating of 4.8 or above, 10+ stays, <1% cancellation rate, and a 90% response rate. Consequently, the criterion of being a superhost implicitly accounts for further characteristics. Paying attention to a high quality standard of the offering and reviews/responses respectively, homeowners should be well-equipped to gain the superhost status. Although the analysis was limited to listings with less than 31 minimum nights, this variable is negatively correlated with the price. Reducing the minimum nights would enable homeowners to exploit the WTP of customers further. This goes hand in hand with the focus on touristic travel which usually stay in one place for less than a week (e.g. when visiting multiple cities in a short time during a road trip). Tourists are generally willing to pay a price premium for their accommodation. This surplus can be capture better when the minimum nights are reduced. 

For Airbnb, we identified review rewards as the most important measure to increase customer engagement and indirectly through that also the willingness to pay of other customers that comes with a larger number of reviews. As described above, encouraging reviews has positive spillover effects via the superhost feature. Moreover, hosts are able to improve their service when they receive adequate feedback and suggestions. This in turn, improves the reputation of Airbnb as a substitute for normal hotels. The incentive scheme could, for example, entail a reduction in the fees that Airbnb charges for the stay if the visitor has a >90% review rate. While the proximity to subways did not significantly increase the price, they should introduce location-specific perks to target home owners in expensive districts. There is a variety of different possibilities for the exact perks to be included in the system. For example, Airbnb could offer a centrally organized cleaning service for the homeowners, which is cheaper than what landlords would regularly pay. As Manhatten has a high density of Airbnbs with short distances, this cleaning service would come at low organizational costs. Lastly, we recommend Airbnb to give homeowner guidance regarding a target price that incorporates all additional factors considered in this analysis. As of now, Airbnb recommends landlords a price based on what other homeowners in that neighbourhood charge. The preceded analysis showed, however, that prices are only to a very small extent rooted on the location. Instead, other characteristics like the room type, host information, and minimum nights are further relevant aspects. As an action plan, we recommend that for each new listing, Airbnb utilizes the coefficients and relative feature weights as estimated in the regressions above to indicate a realistic price based on all the information entered by the homeowner. With that, Airbnb ensures that homeowners optimize price setting based on all information available. 

We want to emphasize again that these recommendations are subject to individual preferences and based entirely on the assumption of profit maximization. They do not account for the preference to host guests for a longer time or to live together with the guest for the stay.